In [1]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.marchtee.com/tees')

html = driver.page_source

soup = BeautifulSoup(html, 'lxml')

In [4]:
product_links = []
for i in soup.find_all("a", class_='absolute-block card-link'):
    product_links.append(i["href"])

In [5]:
product_details_json =[]
for link in product_links:
    if "/long" in link:
        end_link = link.replace("/long","") + ('/long')
    else:
        end_link = link + ('/short')
        
    url = f"https://www.marchtee.com/products{end_link}"
    
    payload = {}
    headers = {
      'accept': 'application/json, text/plain, */*',
      'accept-language': 'en-US,en;q=0.6',
      'cookie': 'style=men; marchtee=.eJxtUm1v2yAQ_iuIT61m2RC7fuunrI3aqGtWJa6qaZksAucElYCHbaVZlP8-nKSNNlUIbJ4XjrtjhynOd5h31oLmW5xjqS32PoCy2a4XRjl83g3IInOUkA03nW5LKRqc_6SDxKOD8JeH4a0Fq5lyhNMnnNM0ywhEIslgkVKappAMBAPBWQosjmOSAskGGclEGNJIhLFIFoKHnJMooi7Sqm3r0kIFFqw7sd82eRBsNht_acxSgc_NOjgLf3fQtE64m-P7ongqn2ejaTm8G02KOc7RHD-aP1IpFlz5BF28SC3MpkGTAlHik2vkgDi6Rm9xdImGda3gBRYPsg2uwsQPY3TxcF88fvOQkq-A7oC_mkt0s7JmDYErgU_6gWasYlaeLHPsuaDjp3J4ezsdzWbHS1AS-jSM3CS-K9BRdPP9eVJMfxwVYy0kO5lnT0fsqzVMoAm06MTuXdr1ymgoD82wW_cV4JL_kvWla0ApsO_coa3OdiY6q8pGdcsD08MrWddSLz9z_Ee9BxpPPiMrxZaH1yLSkLtVAJz_q7C3tMbCP3EO4Fb1h65B472Hy8pCs8J5xVQDbluDXTMN2nW3tR18pNG_NLr_C0q04L0.ZsbrXw.dwk2bcGkRlpap4-FazW9UVjjrxU; product_data=[{"henley":{"men":"XS"}},{"tee":{"men":"3XL"}}]; marchtee=.eJxtUm1v2yAQ_iuIT61m2RC7fuunrI3aqGtWJa6qaZksAucElYCHbaVZlP8-nKSNNlUIbJ4XjrtjhynOd5h31oLmW5xjqS32PoCy2a4XRjl83g3IInOUkA03nW5LKRqc_6SDxKOD8JeH4a0Fq5lyhNMnnNM0ywhEIslgkVKappAMBAPBWQosjmOSAskGGclEGNJIhLFIFoKHnJMooi7Sqm3r0kIFFqw7sd82eRBsNht_acxSgc_NOjgLf3fQtE64m-P7ongqn2ejaTm8G02KOc7RHD-aP1IpFlz5BF28SC3MpkGTAlHik2vkgDi6Rm9xdImGda3gBRYPsg2uwsQPY3TxcF88fvOQkq-A7oC_mkt0s7JmDYErgU_6gWasYlaeLHPsuaDjp3J4ezsdzWbHS1AS-jSM3CS-K9BRdPP9eVJMfxwVYy0kO5lnT0fsqzVMoAm06MTuXdr1ymgoD82wW_cV4JL_kvWla0ApsO_coa3OdiY6q8pGdcsD08MrWddSLz9z_Ee9BxpPPiMrxZaH1yLSkLtVAJz_q7C3tMbCP3EO4Fb1h65B472Hy8pCs8J5xVQDbluDXTMN2nW3tR18pNG_NLr_C0q04L0.Zsbr9A.qxk3FT77i0JLzjT0xGmmlVps3W8',
      'priority': 'u=1, i',
      'referer': 'https://www.marchtee.com/tee/men/navy',
      'sec-ch-ua': '"Not)A;Brand";v="99", "Brave";v="127", "Chromium";v="127"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'sec-fetch-dest': 'empty',
      'sec-fetch-mode': 'cors',
      'sec-fetch-site': 'same-origin',
      'sec-gpc': '1',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    
    product_details_json.append(response.text)


In [6]:
product_id=[]
main_title=[]
brand=[]
category=[]
color=[]
sizes=[]
actualPrice=[]
currentPrice=[]
status=[]

count=1
for tee, link in zip(product_details_json, product_links):
    
    product_detail = json.loads(tee)
    sizes_json = product_detail['products']
    group_info = product_detail['group_info']
    
    for size in sizes_json:
        
        product_id.append(f'MT{count}')
        main_title.append(group_info["description"].replace("<br>"," "))
        brand.append("MarchTee")
        category.append(link.split('/')[1])
        color.append(group_info['id'])
        sizes.append(size['size'])
        actualPrice.append(size['original_price'])
        currentPrice.append(size['price'])
        
        if size['out_of_stock']:
            status.append('OUT_OF_STOCK')
        else:
            status.append("AVAILABLE")
    
    count+=1
    
    
df = pd.DataFrame(
    {
        'product_id':product_id,
        'title':main_title,
        'brand':brand,
        'category':category,
        'color':color,
        'size':sizes,
        'actual_price':actualPrice,
        'current_price':currentPrice,
        'status':status
    }
)


df.head()

,product_id,title,brand,category,color,size,actual_price,current_price,status
0,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,XS,"₹1,800","₹1,500",AVAILABLE
1,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,S,"₹1,800","₹1,500",AVAILABLE
2,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,M,"₹1,800","₹1,500",AVAILABLE
3,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,L,"₹1,800","₹1,500",AVAILABLE
4,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,XL,"₹1,800","₹1,500",AVAILABLE


In [7]:
df

,product_id,title,brand,category,color,size,actual_price,current_price,status
0,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,XS,"₹1,800","₹1,500",AVAILABLE
1,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,S,"₹1,800","₹1,500",AVAILABLE
2,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,M,"₹1,800","₹1,500",AVAILABLE
3,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,L,"₹1,800","₹1,500",AVAILABLE
4,MT1,Midnight Navy 100% Supima Cotton Invisible Sti...,MarchTee,tee,midnight-navy,XL,"₹1,800","₹1,500",AVAILABLE
...,...,...,...,...,...,...,...,...,...
221,MT30,100% Suvin Cotton Extra-long Staple Single Origin,MarchTee,suvin-v,pure-white,S,"₹4,900","₹4,900",OUT_OF_STOCK
222,MT30,100% Suvin Cotton Extra-long Staple Single Origin,MarchTee,suvin-v,pure-white,M,"₹4,900","₹4,900",OUT_OF_STOCK
223,MT30,100% Suvin Cotton Extra-long Staple Single Origin,MarchTee,suvin-v,pure-white,L,"₹4,900","₹4,900",OUT_OF_STOCK
224,MT30,100% Suvin Cotton Extra-long Staple Single Origin,MarchTee,suvin-v,pure-white,XL,"₹4,900","₹4,900",OUT_OF_STOCK


In [8]:
df.to_csv("MT.csv",index=False)